## I. Load & Process the data from Sleeper

In [5]:
from sleeper_functions import *
week = 4
matchup_df, updated_standings, HT_Owner, HT_Score, HP_Owner, HP_Player, HP_Score = run_sleeper_weekly(week)

Loaded the League Data from Sleeper
 - Week 1 has been processed
 - Week 2 has been processed
 - Week 3 has been processed
 - Week 4 has been processed
Completed processing scores and updating standings
Saved week 4 matchup data


## II. Feed Data into GPT

In [2]:
summary = generate_summary(week, matchup_df, updated_standings)
print(summary)

Generated the summary
Greetings Fantasy Gladiators,

In the ceaseless gridiron battleground that is our league, Week 4 brought wins, losses, stinging humiliation, and of course some utterly confounding performances.

Our Top Scorer of the Week honour is bestowed upon the terrifically unpredictable Kai, whose team "Not Sketch" brilliantly danced to victory with a whopping 153.54 points. If someone could explain to me the occult knowledge he consults when picking his team, I'm sure we would all be grateful. According to our logs, Christian McCaffrey had another superhero outing, racking up a ludicrous 45.2 points. On behalf of all fantasy owners who don't have him, Kai, we're not jealous, just emotionally wounded.

In the "winning ugly" category, Johnny's "ChubbHub" bested Peter's "Trust the Process" by a grand total of 3.5 points. Not every victory is a symphony, eh Johnny?

Shoutout to Justin Herbert's 24.38 points and Stefon Diggs' 33 points; it was a valiant effort, but alas, Henry's

## III. Draft Summary Email via Smtpblib

In [6]:
#Load Credentials
import json
# Load the configuration from config.json
with open('config.json') as config_file:
    config_data = json.load(config_file)
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# Load the user name and password from yaml file
user, password = config_data['gmail']['GMAIL_USER'], config_data['gmail']['GMAIL_PW']

#Convert tables to html format
matchup = matchup_df.to_html(index=False)
standings = updated_standings.to_html(index=False)

# Initialize the server
s = smtplib.SMTP('smtp.gmail.com', 587)
s.starttls()
s.login(user, password)

# Define message
subject = f"Dynasty FF Week {week} Recap"
summary_html = summary.replace('\n', '<br>')
body = f"""\
<html>
  <head></head>
  <body>
    <h1>{subject}</h1>
    <br>{summary_html}</br>
    <br>
    <hr>
    <p>Highest Scoring Team: {HT_Owner}: {HT_Score} points</p>
    <p>Highest Scoring Player: {HP_Owner} - {HP_Player}: {HP_Score} Points</p>
    <br>
    {standings}
  </body>
</html>
"""

recipients = list(config_data['emails']['sleeper_email_dict'].values())
sender = "peteryushunli@gmail.com"

msg = MIMEMultipart("alternative")
msg["From"] = sender
msg["To"] = ", ".join(recipients)
msg["Subject"] = subject

# Attach the HTML body
msg.attach(MIMEText(body, 'html'))

# Send email
s.sendmail(sender, recipients, msg.as_string())

# Terminate the session
s.quit()

(221,
 b'2.0.0 closing connection o1-20020ac86d01000000b004199c98f87dsm448068qtt.74 - gsmtp')